In [ ]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual filename)
file_path = "D:\Brain_siez_det\CSV_Output\eeg1.csv"
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

# Check basic info (data types, missing values, etc.)
print(df.info())


In [ ]:
from scipy.signal import butter, filtfilt

# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, data)

# Function to apply lowpass filter
def lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype="low")
    return filtfilt(b, a, data)

# Sampling frequency (estimate from time column)
fs = 1 / (df["time"][1] - df["time"][0])  # Assuming uniform time steps

# Apply filters
for col in df.columns[1:20]:  # EEG and ECG channels
    df[col] = bandpass_filter(df[col], lowcut=0.5, highcut=50, fs=fs)

df["ECG EKG-REF"] = bandpass_filter(df["ECG EKG-REF"], lowcut=0.5, highcut=100, fs=fs)
df["Resp Effort-REF"] = lowpass_filter(df["Resp Effort-REF"], cutoff=1, fs=fs)

print("Filtering complete!")

In [ ]:
import cupy as cp
import pandas as pd
import scipy.stats as stats
import antropy as ant
import time

# Move Signal to GPU
def to_gpu(signal):
    return cp.asarray(signal)

# Move Back to CPU
def to_cpu(value):
    return value.get() if isinstance(value, cp.ndarray) else value

# Compute Shannon Entropy (GPU)
def shannon_entropy(signal):
    signal = to_gpu(signal)
    prob_distribution = cp.histogram(signal, bins=10, density=True)[0]
    prob_distribution = prob_distribution[prob_distribution > 0]
    return to_cpu(-cp.sum(prob_distribution * cp.log2(prob_distribution)))

# Compute Hjorth Parameters (GPU)
def hjorth_parameters(signal):
    signal = to_gpu(signal)
    activity = cp.var(signal)
    diff_signal = cp.diff(signal)
    mobility = cp.sqrt(cp.var(diff_signal) / activity) if activity != 0 else 0
    diff_diff_signal = cp.diff(diff_signal)
    complexity = cp.sqrt(cp.var(diff_diff_signal) / cp.var(diff_signal)) if cp.var(diff_signal) != 0 else 0
    return to_cpu(activity), to_cpu(mobility), to_cpu(complexity)

# Compute Approximate Entropy (CPU)
def approximate_entropy(signal, m=2):
    return ant.app_entropy(signal, order=m, metric='chebyshev')

# Compute Sample Entropy (CPU)
def sample_entropy(signal, m=2):
    return ant.sample_entropy(signal, order=m)

#Compute Permutation Entropy (CPU)
def permutation_entropy(signal, m=3):
    return ant.perm_entropy(signal, order=m)


feature_dict = {}

start_time = time.time()

# Convert full DataFrame to GPU for speedup
gpu_df = {col: to_gpu(df[col].values) for col in df.columns[1:]}

for col, signal in gpu_df.items():
    #  Compute GPU Features
    mean_val = to_cpu(cp.mean(signal))
    std_val = to_cpu(cp.std(signal))
    kurtosis_val = stats.kurtosis(to_cpu(signal))  # SciPy still on CPU
    skewness_val = stats.skew(to_cpu(signal))  # SciPy still on CPU
    shannon_val = shannon_entropy(signal)

    #  Compute Hjorth Parameters (GPU)
    activity, mobility, complexity = hjorth_parameters(signal)

    #  Compute Entropy Features (CPU)
    perm_entropy = permutation_entropy(to_cpu(signal))
    samp_entropy = sample_entropy(to_cpu(signal))
    approx_entropy = approximate_entropy(to_cpu(signal))

    #  Store in Dictionary
    feature_dict[col] = [mean_val, std_val, kurtosis_val, skewness_val, shannon_val,
                         activity, mobility, complexity, perm_entropy, samp_entropy, approx_entropy]

end_time = time.time()
print(f"⚡ GPU-Accelerated Feature Extraction Completed in {end_time - start_time:.2f} seconds!")

#  Convert to DataFrame
feature_df = pd.DataFrame.from_dict(feature_dict, orient="index",
                                    columns=["Mean", "Std Dev", "Kurtosis", "Skewness", "Shannon Entropy",
                                             "Activity", "Mobility", "Complexity",
                                             "Permutation Entropy", "Sample Entropy", "Approximate Entropy"])

#  Display Extracted Features
print(feature_df)


In [ ]:
# Save the feature DataFrame to a CSV file
feature_df.to_csv("extracted_features.csv", index=True)

print("Feature extraction completed and saved to extracted_features.csv!")
